In [1]:
pwd


'/Volumes/data/Kaggle/marchmadness'

In [1]:

import pandas as pd
from fuzzywuzzy import process, fuzz
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [2]:
teams = pd.read_csv("./Teams.csv", index_col=0)

In [3]:
teams.head()

,Team_Name
Team_Id,
1101,Abilene Chr
1102,Air Force
1103,Akron
1104,Alabama
1105,Alabama A&M


In [5]:
print(teams[teams['Team_Name']=='Georgia Tech'])

            Team_Name
Team_Id              
1210     Georgia Tech


In [85]:
print(teams[teams['Team_Id']==1455])

     Team_Id   Team_Name
354     1455  Wichita St


In [6]:
rscr = pd.read_csv("./RegularSeasonCompactResults.csv")

In [7]:
rsdr = pd.read_csv("./RegularSeasonDetailedResults.csv")

In [8]:
tcr = pd.read_csv("./TourneyCompactResults.csv")

In [3]:
ss = pd.read_csv("./sample_submission.csv")
kp = pd.read_csv("./kenpom.csv")

In [10]:
rscr.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [11]:
rscr.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
145284,2016,132,1114,70,1419,50,N,0
145285,2016,132,1163,72,1272,58,N,0
145286,2016,132,1246,82,1401,77,N,1
145287,2016,132,1277,66,1345,62,N,0
145288,2016,132,1386,87,1433,74,N,0


In [12]:
rsdr.head().transpose()

,0,1,2,3,4
Season,2003,2003,2003,2003,2003
Daynum,10,10,11,11,11
Wteam,1104,1272,1266,1296,1400
Wscore,68,70,73,56,77
Lteam,1328,1393,1437,1457,1208
Lscore,62,63,61,50,71
Wloc,N,N,N,N,N
Numot,0,0,0,0,0
Wfgm,27,26,24,18,30
Wfga,58,62,58,38,61


In [13]:
rsdr.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
76631,2017,132,1276,71,1458,56,N,0,27,48,...,15,6,8,14,18,10,15,4,3,13
76632,2017,132,1343,71,1463,59,N,0,25,52,...,20,13,19,14,20,12,7,4,5,13
76633,2017,132,1348,70,1433,63,N,0,24,54,...,14,17,22,23,24,8,5,4,1,16
76634,2017,132,1374,71,1153,56,N,0,26,52,...,24,14,18,17,22,7,7,7,1,13
76635,2017,132,1407,59,1402,53,N,0,21,60,...,17,7,8,9,27,10,17,1,7,18


In [14]:
top_10_reg = teams.join(pd.DataFrame(rscr['Wteam'].value_counts()[:9]), how='right')
top_10_reg.columns = ['Team_Name', 'Total_Wins']
top_10_reg

,Team_Name,Total_Wins
1181,Duke,819
1242,Kansas,804
1246,Kentucky,765
1314,North Carolina,761
1112,Arizona,746
1393,Syracuse,738
1272,Memphis,692
1424,UNLV,685
1257,Louisville,682


In [15]:
ss.head()

,id,pred
0,2013_1103_1107,0.5
1,2013_1103_1112,0.5
2,2013_1103_1125,0.5
3,2013_1103_1129,0.5
4,2013_1103_1137,0.5


The feature we are trying to predict is the probability that team1 will win. In order to do this effectively we need to choose or engineer numerical features that we'll know prior to the game occurring. The makeup of each team varies greatly from year to year. With that in mind I've decided to isolate my train and test data by year. 

In [16]:
rs2013 = rsdr[rsdr['Season']==2013]
rs2014 = rsdr[rsdr['Season']==2014]
rs2015 = rsdr[rsdr['Season']==2015]
rs2016 = rsdr[rsdr['Season']==2016]
rs2017 = rsdr[rsdr['Season']==2017]

In [17]:
rs2013_d1 = pd.DataFrame()
rs2014_d1 = pd.DataFrame()
rs2015_d1 = pd.DataFrame()
rs2016_d1 = pd.DataFrame()
rs2017_d1 = pd.DataFrame()

In [18]:
rs2013_d2 = pd.DataFrame()
rs2014_d2 = pd.DataFrame()
rs2015_d2 = pd.DataFrame()
rs2016_d2 = pd.DataFrame()
rs2017_d2 = pd.DataFrame()

In [19]:
rs2013_d1[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2013[['Wteam', 'Lteam', 'Daynum',  'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk']].copy()
rs2014_d1[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2014[['Wteam', 'Lteam', 'Daynum',  'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk']].copy()
rs2015_d1[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2015[['Wteam', 'Lteam', 'Daynum',  'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk']].copy()
rs2016_d1[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2016[['Wteam', 'Lteam', 'Daynum',  'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk']].copy()
rs2017_d1[['team1', 'team2', 'Daynum']] = rs2017[['Wteam', 'Lteam', 'Daynum']].copy()

In [20]:
rs2013_d2[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2013[['Lteam', 'Wteam', 'Daynum', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk']].copy()
rs2014_d2[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2014[['Lteam', 'Wteam', 'Daynum', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk']].copy()
rs2015_d2[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2015[['Lteam', 'Wteam', 'Daynum', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk']].copy()
rs2016_d2[['team1', 'team2', 'Daynum', 't1_fgm', 't1_fga', 't1_fgm3', 't1_fga3', 't1_ftm', 't1_fta', 't1_ast', 't1_dr', 't1_stl', 't1_pf', 't1_blk', 't2_fgm', 't2_fga', 't2_fgm3', 't2_fga3', 't2_ftm', 't2_fta', 't2_ast', 't2_dr', 't2_stl', 't2_pf', 't2_blk' ]] = rs2016[['Lteam', 'Wteam', 'Daynum', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3', 'Lftm', 'Lfta', 'Last', 'Ldr', 'Lstl', 'Lpf', 'Lblk', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wast', 'Wdr', 'Wstl', 'Wpf', 'Wblk']].copy()
rs2017_d2[['team1', 'team2', 'Daynum']] = rs2017[['Lteam', 'Wteam', 'Daynum']].copy()

In [21]:
rs2013_d1['pred'] = 1
rs2014_d1['pred'] = 1
rs2015_d1['pred'] = 1
rs2016_d1['pred'] = 1
rs2017_d1['pred'] = 1

In [22]:
rs2013_d2['pred'] = 0
rs2014_d2['pred'] = 0
rs2015_d2['pred'] = 0
rs2016_d2['pred'] = 0
rs2017_d2['pred'] = 0

In [23]:
rs2013_train = pd.concat([rs2013_d1, rs2013_d2]).reset_index(drop=True)
rs2014_train = pd.concat([rs2014_d1, rs2014_d2]).reset_index(drop=True)
rs2015_train = pd.concat([rs2015_d1, rs2015_d2]).reset_index(drop=True)
rs2016_train = pd.concat([rs2016_d1, rs2016_d2]).reset_index(drop=True)
rs2017_train = pd.concat([rs2017_d1, rs2017_d2]).reset_index(drop=True)

In [24]:
rs2013_train['t1_fg_pct'] = rs2013_train['t1_fgm'] / rs2013_train['t1_fga']
rs2013_train['t1_fg3_pct'] = rs2013_train['t1_fgm3'] / rs2013_train['t1_fga3']
rs2013_train['t1_ft_pct'] = rs2013_train['t1_ftm'] / rs2013_train['t1_fta']
rs2014_train['t1_fg_pct'] = rs2014_train['t1_fgm'] / rs2014_train['t1_fga']
rs2014_train['t1_fg3_pct'] = rs2014_train['t1_fgm3'] / rs2014_train['t1_fga3']
rs2014_train['t1_ft_pct'] = rs2014_train['t1_ftm'] / rs2014_train['t1_fta']
rs2015_train['t1_fg_pct'] = rs2015_train['t1_fgm'] / rs2015_train['t1_fga']
rs2015_train['t1_fg3_pct'] = rs2015_train['t1_fgm3'] / rs2015_train['t1_fga3']
rs2015_train['t1_ft_pct'] = rs2015_train['t1_ftm'] / rs2015_train['t1_fta']
rs2016_train['t1_fg_pct'] = rs2016_train['t1_fgm'] / rs2016_train['t1_fga']
rs2016_train['t1_fg3_pct'] = rs2016_train['t1_fgm3'] / rs2016_train['t1_fga3']
rs2016_train['t1_ft_pct'] = rs2016_train['t1_ftm'] / rs2016_train['t1_fta']

In [25]:
rs2013_train['t2_fg_pct'] = rs2013_train['t2_fgm'] / rs2013_train['t2_fga']
rs2013_train['t2_fg3_pct'] = rs2013_train['t2_fgm3'] / rs2013_train['t2_fga3']
rs2013_train['t2_ft_pct'] = rs2013_train['t2_ftm'] / rs2013_train['t2_fta']
rs2014_train['t2_fg_pct'] = rs2014_train['t2_fgm'] / rs2014_train['t2_fga']
rs2014_train['t2_fg3_pct'] = rs2014_train['t2_fgm3'] / rs2014_train['t2_fga3']
rs2014_train['t2_ft_pct'] = rs2014_train['t2_ftm'] / rs2014_train['t2_fta']
rs2015_train['t2_fg_pct'] = rs2015_train['t2_fgm'] / rs2015_train['t2_fga']
rs2015_train['t2_fg3_pct'] = rs2015_train['t2_fgm3'] / rs2015_train['t2_fga3']
rs2015_train['t2_ft_pct'] = rs2015_train['t2_ftm'] / rs2015_train['t2_fta']
rs2016_train['t2_fg_pct'] = rs2016_train['t2_fgm'] / rs2016_train['t2_fga']
rs2016_train['t2_fg3_pct'] = rs2016_train['t2_fgm3'] / rs2016_train['t2_fga3']
rs2016_train['t2_ft_pct'] = rs2016_train['t2_ftm'] / rs2016_train['t2_fta']

In [26]:
rs2013_train = rs2013_train.sort_values('Daynum')
rs2014_train = rs2013_train.sort_values('Daynum')
rs2015_train = rs2013_train.sort_values('Daynum')
rs2016_train = rs2013_train.sort_values('Daynum')

In [27]:
t1_fg_pct_avg_2013 = rs2013_train.groupby('team1', as_index=False)['t1_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_fg_pct_avg'] = t1_fg_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t1_fg3_pct_avg_2013 = rs2013_train.groupby('team1', as_index=False)['t1_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_fg3_pct_avg'] = t1_fg3_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t1_ft_pct_avg_2013 = rs2013_train.groupby('team1', as_index=False)['t1_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_ft_pct_avg'] = t1_ft_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t1_ast_2013 = rs2013_train.groupby('team1', as_index=False)['t1_ast'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_ast_avg'] = t1_ast_2013.fillna(0).reset_index(level=0, drop=True)

t1_dr_2013 = rs2013_train.groupby('team1', as_index=False)['t1_dr'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_dr_avg'] = t1_dr_2013.fillna(0).reset_index(level=0, drop=True)

t1_stl_2013 = rs2013_train.groupby('team1', as_index=False)['t1_stl'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_stl_avg'] = t1_stl_2013.fillna(0).reset_index(level=0, drop=True)

t1_pf_2013 = rs2013_train.groupby('team1', as_index=False)['t1_pf'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_pf_avg'] = t1_pf_2013.fillna(0).reset_index(level=0, drop=True)

t1_blk_2013 = rs2013_train.groupby('team1', as_index=False)['t1_blk'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t1_blk_avg'] = t1_blk_2013.fillna(0).reset_index(level=0, drop=True)

In [28]:
t1_fg_pct_avg_2014 = rs2014_train.groupby('team1', as_index=False)['t1_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_fg_pct_avg'] = t1_fg_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t1_fg3_pct_avg_2014 = rs2014_train.groupby('team1', as_index=False)['t1_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_fg3_pct_avg'] = t1_fg3_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t1_ft_pct_avg_2014 = rs2014_train.groupby('team1', as_index=False)['t1_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_ft_pct_avg'] = t1_ft_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t1_ast_2014 = rs2014_train.groupby('team1', as_index=False)['t1_ast'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_ast_avg'] = t1_ast_2014.fillna(0).reset_index(level=0, drop=True)

t1_dr_2014 = rs2014_train.groupby('team1', as_index=False)['t1_dr'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_dr_avg'] = t1_dr_2014.fillna(0).reset_index(level=0, drop=True)

t1_stl_2014 = rs2014_train.groupby('team1', as_index=False)['t1_stl'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_stl_avg'] = t1_stl_2014.fillna(0).reset_index(level=0, drop=True)

t1_pf_2014 = rs2014_train.groupby('team1', as_index=False)['t1_pf'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_pf_avg'] = t1_pf_2014.fillna(0).reset_index(level=0, drop=True)

t1_blk_2014 = rs2014_train.groupby('team1', as_index=False)['t1_blk'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t1_blk_avg'] = t1_blk_2014.fillna(0).reset_index(level=0, drop=True)

In [29]:
t1_fg_pct_avg_2015 = rs2015_train.groupby('team1', as_index=False)['t1_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_fg_pct_avg'] = t1_fg_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t1_fg3_pct_avg_2015 = rs2015_train.groupby('team1', as_index=False)['t1_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_fg3_pct_avg'] = t1_fg3_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t1_ft_pct_avg_2015 = rs2015_train.groupby('team1', as_index=False)['t1_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_ft_pct_avg'] = t1_ft_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t1_ast_2015 = rs2015_train.groupby('team1', as_index=False)['t1_ast'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_ast_avg'] = t1_ast_2015.fillna(0).reset_index(level=0, drop=True)

t1_dr_2015 = rs2015_train.groupby('team1', as_index=False)['t1_dr'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_dr_avg'] = t1_dr_2015.fillna(0).reset_index(level=0, drop=True)

t1_stl_2015 = rs2015_train.groupby('team1', as_index=False)['t1_stl'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_stl_avg'] = t1_stl_2015.fillna(0).reset_index(level=0, drop=True)

t1_pf_2015 = rs2015_train.groupby('team1', as_index=False)['t1_pf'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_pf_avg'] = t1_pf_2015.fillna(0).reset_index(level=0, drop=True)

t1_blk_2015 = rs2015_train.groupby('team1', as_index=False)['t1_blk'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t1_blk_avg'] = t1_blk_2015.fillna(0).reset_index(level=0, drop=True)

In [30]:
t1_fg_pct_avg_2016 = rs2016_train.groupby('team1', as_index=False)['t1_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_fg_pct_avg'] = t1_fg_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t1_fg3_pct_avg_2016 = rs2016_train.groupby('team1', as_index=False)['t1_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_fg3_pct_avg'] = t1_fg3_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t1_ft_pct_avg_2016 = rs2016_train.groupby('team1', as_index=False)['t1_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_ft_pct_avg'] = t1_ft_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t1_ast_2016 = rs2016_train.groupby('team1', as_index=False)['t1_ast'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_ast_avg'] = t1_ast_2016.fillna(0).reset_index(level=0, drop=True)

t1_dr_2016 = rs2016_train.groupby('team1', as_index=False)['t1_dr'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_dr_avg'] = t1_dr_2016.fillna(0).reset_index(level=0, drop=True)

t1_stl_2016 = rs2016_train.groupby('team1', as_index=False)['t1_stl'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_stl_avg'] = t1_stl_2016.fillna(0).reset_index(level=0, drop=True)

t1_pf_2016 = rs2016_train.groupby('team1', as_index=False)['t1_pf'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_pf_avg'] = t1_pf_2016.fillna(0).reset_index(level=0, drop=True)

t1_blk_2016 = rs2016_train.groupby('team1', as_index=False)['t1_blk'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t1_blk_avg'] = t1_blk_2016.fillna(0).reset_index(level=0, drop=True)

In [31]:
t2_fg_pct_avg_2013 = rs2013_train.groupby('team2', as_index=False)['t2_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_fg_pct_avg'] = t2_fg_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t2_fg3_pct_avg_2013 = rs2013_train.groupby('team2', as_index=False)['t2_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_fg3_pct_avg'] = t2_fg3_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t2_ft_pct_avg_2013 = rs2013_train.groupby('team2', as_index=False)['t2_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_ft_pct_avg'] = t2_ft_pct_avg_2013.fillna(0).reset_index(level=0, drop=True)

t2_ast_2013 = rs2013_train.groupby('team2', as_index=False)['t2_ast'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_ast_avg'] = t2_ast_2013.fillna(0).reset_index(level=0, drop=True)

t2_dr_2013 = rs2013_train.groupby('team2', as_index=False)['t2_dr'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_dr_avg'] = t2_dr_2013.fillna(0).reset_index(level=0, drop=True)

t2_stl_2013 = rs2013_train.groupby('team2', as_index=False)['t2_stl'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_stl_avg'] = t2_stl_2013.fillna(0).reset_index(level=0, drop=True)

t2_pf_2013 = rs2013_train.groupby('team2', as_index=False)['t2_pf'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_pf_avg'] = t2_pf_2013.fillna(0).reset_index(level=0, drop=True)

t2_blk_2013 = rs2013_train.groupby('team2', as_index=False)['t2_blk'].apply(lambda x: x.expanding().mean().shift())
rs2013_train['t2_blk_avg'] = t2_blk_2013.fillna(0).reset_index(level=0, drop=True)



In [32]:
t2_fg_pct_avg_2014 = rs2014_train.groupby('team2', as_index=False)['t2_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_fg_pct_avg'] = t2_fg_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t2_fg3_pct_avg_2014 = rs2014_train.groupby('team2', as_index=False)['t2_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_fg3_pct_avg'] = t2_fg3_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t2_ft_pct_avg_2014 = rs2014_train.groupby('team2', as_index=False)['t2_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_ft_pct_avg'] = t2_ft_pct_avg_2014.fillna(0).reset_index(level=0, drop=True)

t2_ast_2014 = rs2014_train.groupby('team2', as_index=False)['t2_ast'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_ast_avg'] = t2_ast_2014.fillna(0).reset_index(level=0, drop=True)

t2_dr_2014 = rs2014_train.groupby('team2', as_index=False)['t2_dr'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_dr_avg'] = t2_dr_2014.fillna(0).reset_index(level=0, drop=True)

t2_stl_2014 = rs2014_train.groupby('team2', as_index=False)['t2_stl'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_stl_avg'] = t2_stl_2014.fillna(0).reset_index(level=0, drop=True)

t2_pf_2014 = rs2014_train.groupby('team2', as_index=False)['t2_pf'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_pf_avg'] = t2_pf_2014.fillna(0).reset_index(level=0, drop=True)

t2_blk_2014 = rs2014_train.groupby('team2', as_index=False)['t2_blk'].apply(lambda x: x.expanding().mean().shift())
rs2014_train['t2_blk_avg'] = t2_blk_2014.fillna(0).reset_index(level=0, drop=True)



In [33]:
t2_fg_pct_avg_2015 = rs2015_train.groupby('team2', as_index=False)['t2_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_fg_pct_avg'] = t2_fg_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t2_fg3_pct_avg_2015 = rs2015_train.groupby('team2', as_index=False)['t2_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_fg3_pct_avg'] = t2_fg3_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t2_ft_pct_avg_2015 = rs2015_train.groupby('team2', as_index=False)['t2_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_ft_pct_avg'] = t2_ft_pct_avg_2015.fillna(0).reset_index(level=0, drop=True)

t2_ast_2015 = rs2015_train.groupby('team2', as_index=False)['t2_ast'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_ast_avg'] = t2_ast_2015.fillna(0).reset_index(level=0, drop=True)

t2_dr_2015 = rs2015_train.groupby('team2', as_index=False)['t2_dr'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_dr_avg'] = t2_dr_2015.fillna(0).reset_index(level=0, drop=True)

t2_stl_2015 = rs2015_train.groupby('team2', as_index=False)['t2_stl'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_stl_avg'] = t2_stl_2015.fillna(0).reset_index(level=0, drop=True)

t2_pf_2015 = rs2015_train.groupby('team2', as_index=False)['t2_pf'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_pf_avg'] = t2_pf_2015.fillna(0).reset_index(level=0, drop=True)

t2_blk_2015 = rs2015_train.groupby('team2', as_index=False)['t2_blk'].apply(lambda x: x.expanding().mean().shift())
rs2015_train['t2_blk_avg'] = t2_blk_2015.fillna(0).reset_index(level=0, drop=True)



In [34]:
t2_fg_pct_avg_2016 = rs2016_train.groupby('team2', as_index=False)['t2_fg_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_fg_pct_avg'] = t2_fg_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t2_fg3_pct_avg_2016 = rs2016_train.groupby('team2', as_index=False)['t2_fg3_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_fg3_pct_avg'] = t2_fg3_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t2_ft_pct_avg_2016 = rs2016_train.groupby('team2', as_index=False)['t2_ft_pct'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_ft_pct_avg'] = t2_ft_pct_avg_2016.fillna(0).reset_index(level=0, drop=True)

t2_ast_2016 = rs2016_train.groupby('team2', as_index=False)['t2_ast'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_ast_avg'] = t2_ast_2016.fillna(0).reset_index(level=0, drop=True)

t2_dr_2016 = rs2016_train.groupby('team2', as_index=False)['t2_dr'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_dr_avg'] = t2_dr_2016.fillna(0).reset_index(level=0, drop=True)

t2_stl_2016 = rs2016_train.groupby('team2', as_index=False)['t2_stl'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_stl_avg'] = t2_stl_2016.fillna(0).reset_index(level=0, drop=True)

t2_pf_2016 = rs2016_train.groupby('team2', as_index=False)['t2_pf'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_pf_avg'] = t2_pf_2016.fillna(0).reset_index(level=0, drop=True)

t2_blk_2016 = rs2016_train.groupby('team2', as_index=False)['t2_blk'].apply(lambda x: x.expanding().mean().shift())
rs2016_train['t2_blk_avg'] = t2_blk_2016.fillna(0).reset_index(level=0, drop=True)



In [35]:
target13 = rs2013_train['pred'].values
target14 = rs2014_train['pred'].values
target15 = rs2015_train['pred'].values
target16 = rs2016_train['pred'].values

In [36]:
features_array13 = rs2013_train[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
features_array14 = rs2014_train[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
features_array15 = rs2015_train[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
features_array16 = rs2016_train[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values

In [37]:
logreg = LogisticRegression(C=1)
lrscores13 = cross_val_score(logreg, features_array13, target13, cv=5)
lrscores14 = cross_val_score(logreg, features_array14, target14, cv=5)
lrscores15 = cross_val_score(logreg, features_array15, target15, cv=5)
lrscores16 = cross_val_score(logreg, features_array16, target16, cv=5)

In [38]:
lrscores13.mean()
lrscores14.mean()
lrscores15.mean()
lrscores16.mean()

0.62546992481203012

In [39]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rfscores13 = cross_val_score(rf, features_array13, target13, cv=5, n_jobs=4, scoring='accuracy')
rfscores14 = cross_val_score(rf, features_array14, target14, cv=5, n_jobs=4, scoring='accuracy')
rfscores15 = cross_val_score(rf, features_array15, target15, cv=5, n_jobs=4, scoring='accuracy')
rfscores16 = cross_val_score(rf, features_array16, target16, cv=5, n_jobs=4, scoring='accuracy')

print('Random Forest CV Scores:')
print('min: {:.3f}, mean; {:.3f}, max: {:.3f}'.format(
      rfscores13.min(), rfscores13.mean(), rfscores13.max()))
print('min: {:.3f}, mean; {:.3f}, max: {:.3f}'.format(
      rfscores14.min(), rfscores14.mean(), rfscores14.max()))
print('min: {:.3f}, mean; {:.3f}, max: {:.3f}'.format(
      rfscores15.min(), rfscores15.mean(), rfscores15.max()))
print('min: {:.3f}, mean; {:.3f}, max: {:.3f}'.format(
      rfscores16.min(), rfscores16.mean(), rfscores16.max()))

Random Forest CV Scores:
min: 0.594, mean; 0.641, max: 0.697
min: 0.605, mean; 0.644, max: 0.706
min: 0.605, mean; 0.644, max: 0.706
min: 0.605, mean; 0.644, max: 0.706


In [40]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, subsample=.8, max_features=.5)

gbscores13 = cross_val_score(gb, features_array13, target13, cv=10, n_jobs=4, scoring='accuracy')
gbscores14 = cross_val_score(gb, features_array14, target14, cv=10, n_jobs=4, scoring='accuracy')
gbscores15 = cross_val_score(gb, features_array15, target15, cv=10, n_jobs=4, scoring='accuracy')
gbscores16 = cross_val_score(gb, features_array16, target16, cv=10, n_jobs=4, scoring='accuracy')

print("Gradient Boosted Trees CV scores:")
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores13.min(), gbscores13.mean(), gbscores13.max()))
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores14.min(), gbscores14.mean(), gbscores14.max()))
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores15.min(), gbscores15.mean(), gbscores15.max()))
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores16.min(), gbscores16.mean(), gbscores16.max()))

Gradient Boosted Trees CV scores:
min: 0.562, mean: 0.640, max: 0.711
min: 0.558, mean: 0.642, max: 0.712
min: 0.558, mean: 0.642, max: 0.712
min: 0.558, mean: 0.642, max: 0.712


For now I'll stick with Gradient Boosting as my classifier and I'll start munging the test data that I'll be using to make my predictions with. First I need to get the sample submission csv that Kaggle gave us into a format that I can use with scikit-learn.

In [41]:
ss['year'] = ss.id.str.split('_').str.get(0)
ss['team1'] = ss.id.str.split('_').str.get(1)
ss['team2'] = ss.id.str.split('_').str.get(2)
ss.drop('pred', axis=1, inplace=True)
ss[['year', 'team1', 'team2']] = ss[['year', 'team1', 'team2']].apply(pd.to_numeric)
ss13 = ss[ss['year']==2013]
ss14 = ss[ss['year']==2014]
ss15 = ss[ss['year']==2015]
ss16 = ss[ss['year']==2016]


In [42]:
rs2013_end = pd.DataFrame()
rs2014_end = pd.DataFrame()
rs2015_end = pd.DataFrame()
rs2016_end = pd.DataFrame()

In [43]:
rs2013_end = rs2013_train.groupby('team1').last().reset_index(drop=False)
rs2014_end = rs2014_train.groupby('team1').last().reset_index(drop=False)
rs2015_end = rs2015_train.groupby('team1').last().reset_index(drop=False)
rs2016_end = rs2016_train.groupby('team1').last().reset_index(drop=False)

In [44]:
t1_2013_end = rs2013_end[['team1', 't1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg']]
ss13 = pd.merge(ss13, t1_2013_end, on='team1')
t2_2013_end = pd.DataFrame()
t2_2013_end[['team2', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']] = t1_2013_end
ss13 = pd.merge(ss13, t2_2013_end, on='team2')

In [45]:
t1_2014_end = rs2014_end[['team1', 't1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg']]
ss14 = pd.merge(ss14, t1_2014_end, on='team1')
t2_2014_end = pd.DataFrame()
t2_2014_end[['team2', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']] = t1_2014_end
ss14 = pd.merge(ss14, t2_2014_end, on='team2')

In [46]:
t1_2015_end = rs2015_end[['team1', 't1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg']]
ss15 = pd.merge(ss15, t1_2015_end, on='team1')
t2_2015_end = pd.DataFrame()
t2_2015_end[['team2', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']] = t1_2015_end
ss15 = pd.merge(ss15, t2_2015_end, on='team2')

In [47]:
t1_2016_end = rs2016_end[['team1', 't1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg']]
ss16 = pd.merge(ss16, t1_2016_end, on='team1')
t2_2016_end = pd.DataFrame()
t2_2016_end[['team2', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']] = t1_2016_end
ss16 = pd.merge(ss16, t2_2016_end, on='team2')

In [48]:
test_13 = ss13[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
test_14 = ss14[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
test_15 = ss15[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values
test_16 = ss16[['t1_fg_pct_avg', 't1_fg3_pct_avg', 't1_ft_pct_avg', 't2_fg_pct_avg', 't2_fg3_pct_avg', 't2_ft_pct_avg', 't1_ast_avg', 't1_dr_avg','t1_stl_avg','t1_pf_avg','t1_blk_avg', 't2_ast_avg', 't2_dr_avg','t2_stl_avg','t2_pf_avg','t2_blk_avg']].values

In [49]:
gb.fit(features_array13, target13)
t1_pred_prob_2013 = gb.predict_proba(test_13)

gb.fit(features_array14, target14)
t1_pred_prob_2014 = gb.predict_proba(test_14)

gb.fit(features_array15, target15)
t1_pred_prob_2015 = gb.predict_proba(test_15)

gb.fit(features_array16, target16)
t1_pred_prob_2016 = gb.predict_proba(test_16)

In [50]:
preds13 = t1_pred_prob_2013[:,1]
ss13['pred'] = preds13
preds14 = t1_pred_prob_2014[:,1]
ss14['pred'] = preds14
preds15 = t1_pred_prob_2015[:,1]
ss15['pred'] = preds15
preds16 = t1_pred_prob_2016[:,1]
ss16['pred'] = preds16

In [51]:
ss13 = ss13[['id', 'pred']]
ss14 = ss14[['id', 'pred']]
ss15 = ss15[['id', 'pred']]
ss16 = ss16[['id', 'pred']]

In [52]:
ss13 = ss13.sort_values('id').reset_index(drop=True)
ss14 = ss14.sort_values('id').reset_index(drop=True)
ss15 = ss15.sort_values('id').reset_index(drop=True)
ss16 = ss16.sort_values('id').reset_index(drop=True)

In [53]:

#ss_3_10_1635 = pd.concat([ss13, ss14, ss15, ss16], ignore_index=True)

In [54]:
#ss_3_10_1635.to_csv('ss_3_10_1635.csv', index=False)

The following section involves using data from Ken Pomeroy. Various publications have indicated that these features are helpful in predicting NCAA matchups.

In [5]:
kp.tail()

,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,...,Luck,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank
3468,2017,347,Longwood,BSth,6,24,NaN,-25.08,92.4,338,...,0.017,144,-5.24,292,102.0,286,107.2,274,0.20,160
3469,2017,348,Arkansas Pine Bluff,SWAC,7,25,NaN,-25.24,85.6,351,...,-0.010,217,-8.83,338,101.0,320,109.8,343,3.22,72
3470,2017,349,North Carolina A&T,MEAC,3,29,NaN,-27.49,95.7,322,...,-0.086,339,-8.75,337,99.3,340,108.0,309,0.50,152
3471,2017,350,Presbyterian,BSth,5,25,NaN,-29.21,89.2,349,...,0.018,136,-3.89,251,103.1,243,107.0,265,2.80,80
3472,2017,351,Alabama A&M,SWAC,2,27,NaN,-30.32,91.6,341,...,-0.075,331,-9.67,343,99.4,338,109.0,337,1.39,127


In [4]:
teams.reset_index(level=0, inplace=True)

In [7]:
teams.tail(5)

,Team_Id,Team_Name
359,1460,Wright St
360,1461,Wyoming
361,1462,Xavier
362,1463,Yale
363,1464,Youngstown St


In [5]:
kp_2017 = kp[kp['Year']==2017]

In [7]:
# kp_2017['Team'] = kp_2017['Team'].apply(lambda x: difflib.get_close_matches(x, teams['Team_Name']))
# kp_id17 = pd.merge(teams, kp_2017, how='left', left_on='Team_Name', right_on='Team')
# kp_id17

/Users/jeorryb/anaconda3/envs/marchmad/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
kp_2017.tail()

,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,...,Luck,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank
3468,2017,347,[Longwood],BSth,6,24,NaN,-25.08,92.4,338,...,0.017,144,-5.24,292,102.0,286,107.2,274,0.20,160
3469,2017,348,"[Ark Pine Bluff, Arkansas St]",SWAC,7,25,NaN,-25.24,85.6,351,...,-0.010,217,-8.83,338,101.0,320,109.8,343,3.22,72
3470,2017,349,"[North Carolina, South Carolina, S Carolina St]",MEAC,3,29,NaN,-27.49,95.7,322,...,-0.086,339,-8.75,337,99.3,340,108.0,309,0.50,152
3471,2017,350,[Presbyterian],BSth,5,25,NaN,-29.21,89.2,349,...,0.018,136,-3.89,251,103.1,243,107.0,265,2.80,80
3472,2017,351,"[Alabama A&M, Alabama, Alabama St]",SWAC,2,27,NaN,-30.32,91.6,341,...,-0.075,331,-9.67,343,99.4,338,109.0,337,1.39,127


In [27]:
kp_id2017 = kp_id2017.sort_values('Team_Id')

In [28]:

kp_id2017.tail()

,Team_Id,Team_Name,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,...,Luck,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank
2062,1458,Wisconsin,2017.0,22.0,Wisconsin,B10,23.0,8.0,NaN,21.51,...,-0.002,191.0,8.40,46.0,109.6,40.0,101.1,51.0,-0.70,190.0
2068,1459,Wofford,2017.0,140.0,Wofford,SC,16.0,17.0,NaN,1.83,...,-0.067,320.0,1.66,102.0,106.1,94.0,104.4,124.0,7.35,24.0
2078,1461,Wyoming,2017.0,147.0,Wyoming,MWC,18.0,14.0,NaN,1.29,...,0.010,167.0,0.44,131.0,105.0,143.0,104.6,135.0,-1.93,221.0
2086,1462,Xavier,2017.0,45.0,Xavier,BE,19.0,12.0,NaN,14.87,...,0.031,101.0,11.13,17.0,111.0,18.0,99.9,20.0,6.00,37.0
2095,1463,Yale,2017.0,146.0,Yale,Ivy,17.0,10.0,NaN,1.33,...,0.046,72.0,-2.30,202.0,103.8,209.0,106.1,206.0,-0.66,186.0


In [62]:
kp_feat_17 = pd.merge(kp_id2017, rs2017_train, left_on='Team_Id', right_on='team1')

In [63]:
kp_feat_17.head()

,Team_Id,Team_Name,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,...,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank,team1,team2,Daynum,pred
0,1102,Air Force,2017,219,Air Force,MWC,12,20,NaN,-4.77,...,104.5,170,104.4,128,-6.02,328,1102,1239,14,1
1,1102,Air Force,2017,219,Air Force,MWC,12,20,NaN,-4.77,...,104.5,170,104.4,128,-6.02,328,1102,1197,20,1
2,1102,Air Force,2017,219,Air Force,MWC,12,20,NaN,-4.77,...,104.5,170,104.4,128,-6.02,328,1102,1391,22,1
3,1102,Air Force,2017,219,Air Force,MWC,12,20,NaN,-4.77,...,104.5,170,104.4,128,-6.02,328,1102,1283,33,1
4,1102,Air Force,2017,219,Air Force,MWC,12,20,NaN,-4.77,...,104.5,170,104.4,128,-6.02,328,1102,1413,52,1


In [64]:
kp_feat_17 = kp_feat_17.rename(columns={'Team_Name':'t1_name', 'Rank':'t1_rank', 'Conference':'t1_conf', 
                           'Wins':'t1_wins', 'Losses':'t1_losses','Pyth':'t1_pyth', 'AdjustO':'t1_adjusto',
                           'AdjustD':'t1_adjustd', 'AdjustT':'t1_adjustt', 'Luck':'t1_luck', 'SOS Pyth':'t1_sospyth',
                           'SOS OppO':'t1_sos_oppo', 'SOS OppD':'t1_sos_oppd', 'NCSOS Pyth':'t1_ncsos_pyth'})

In [65]:
kp_feat_17.drop(['Team_Id', 'Team', 'Daynum'], axis=1, inplace=True)

In [66]:
kp_feat_17 = pd.merge(kp_id2017, kp_feat_17, left_on='Team_Id', right_on='team2')

In [68]:
kp_feat_17 = kp_feat_17.rename(columns={'Team_Name':'t2_name', 'Rank':'t2_rank', 'Conference':'t2_conf', 
                           'Wins':'t2_wins', 'Losses':'t2_losses','Pyth':'t2_pyth', 'AdjustO':'t2_adjusto',
                           'AdjustD':'t2_adjustd', 'AdjustT':'t2_adjustt', 'Luck':'t2_luck', 'SOS Pyth':'t2_sospyth',
                           'SOS OppO':'t2_sos_oppo', 'SOS OppD':'t2_sos_oppd', 'NCSOS Pyth':'t2_ncsos_pyth'})

In [69]:
kp_feat_17.drop(['Seed_x', 'Team', 'AdjustO Rank_x', 'AdjustD Rank_x', 'AdjustT Rank_x', 'Luck Rank_x',
                'SOS Pyth Rank_x', 'SOS OppO Rank_x', 'SOS OppD Rank_x', 'NCSOS Pyth Rank_x', 'Year_y', 'Seed_y', 
                'AdjustO Rank_y', 'AdjustD Rank_y', 'AdjustT Rank_y', 'Luck Rank_y', 'SOS OppO Rank_y',
                'SOS OppD Rank_y', 'NCSOS Pyth Rank_y'], axis=1, inplace=True)

In [70]:
kp_feat_17.drop('Team_Id', axis=1, inplace=True)

In [71]:
target17 = kp_feat_17['pred'].values

In [72]:
features_array17 = kp_feat_17[['t1_ncsos_pyth', 't1_sos_oppd', 't1_sos_oppo', 't1_sospyth', 't1_luck', 't1_adjustt',
                               't1_adjusto', 't1_adjustd', 't1_pyth', 't2_ncsos_pyth', 't2_sos_oppd', 't2_sos_oppo',
                               't2_sospyth', 't2_luck', 't2_adjustt', 't2_adjusto', 't2_adjustd', 't2_pyth']].values

In [73]:
gbscores17 = cross_val_score(gb, features_array17, target17, cv=10, n_jobs=4, scoring='accuracy')

print("Gradient Boosted Trees CV scores:")
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    gbscores17.min(), gbscores17.mean(), gbscores17.max()))

Gradient Boosted Trees CV scores:
min: 0.650, mean: 0.702, max: 0.748


In [9]:
ssub = pd.read_csv("./SampleSubmission.csv")

In [10]:
ssub.head()

,Id,Pred
0,2017_1112_1116,0.5
1,2017_1112_1124,0.5
2,2017_1112_1137,0.5
3,2017_1112_1139,0.5
4,2017_1112_1153,0.5


In [11]:
ssub['year'] = ssub.Id.str.split('_').str.get(0)
ssub['team1'] = ssub.Id.str.split('_').str.get(1)
ssub['team2'] = ssub.Id.str.split('_').str.get(2)
ssub.drop('Pred', axis=1, inplace=True)
ssub[['year', 'team1', 'team2']] = ssub[['year', 'team1', 'team2']].apply(pd.to_numeric)
ss17 = ssub

In [13]:
ss17 = ss17.merge(teams, how='left', left_on='team1', right_on='Team_Id')

In [15]:
ss17.tail()

,Id,year,team1,team2,Team_Id,Team_Name
2273,2017_1455_1458,2017,1455,1458,NaN,NaN
2274,2017_1455_1462,2017,1455,1462,NaN,NaN
2275,2017_1457_1458,2017,1457,1458,NaN,NaN
2276,2017_1457_1462,2017,1457,1462,NaN,NaN
2277,2017_1458_1462,2017,1458,1462,NaN,NaN


In [78]:
kp2017_end = kp_feat_17.groupby('team1').last().reset_index(drop=False)
kp2017_end['team1'] = kp2017_end['team1'].apply(pd.to_numeric)


t1_2017_end = kp2017_end[['team1', 't1_ncsos_pyth', 't1_sos_oppd', 't1_sos_oppo', 't1_sospyth', 't1_luck',
                          't1_adjustt','t1_adjusto', 't1_adjustd', 't1_pyth']]

ss17 = ss17.merge(t1_2017_end, how='left', left_on='team1', right_on='team1')

t2_2017_end = pd.DataFrame()
t2_2017_end[['team2', 't2_ncsos_pyth', 't2_sos_oppd', 't2_sos_oppo', 't2_sospyth', 't2_luck',
                          't2_adjustt','t2_adjusto', 't2_adjustd', 't2_pyth']] = t1_2017_end

ss17 = ss17.merge(t2_2017_end, how='left', left_on='team2', right_on='team2')
ss17.tail()

,Id,year,team1,team2,t1_ncsos_pyth,t1_sos_oppd,t1_sos_oppo,t1_sospyth,t1_luck,t1_adjustt,...,t1_pyth,t2_ncsos_pyth,t2_sos_oppd,t2_sos_oppo,t2_sospyth,t2_luck,t2_adjustt,t2_adjusto,t2_adjustd,t2_pyth
2273,2017_1455_1458,2017,1455,1458,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.7,101.1,109.6,8.40,-0.002,63.6,113.5,92.0,21.51
2274,2017_1455_1462,2017,1455,1462,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.0,99.9,111.0,11.13,0.031,67.8,115.3,100.4,14.87
2275,2017_1457_1458,2017,1457,1458,3.04,108.0,100.7,-7.3,0.029,71.2,...,4.74,-0.7,101.1,109.6,8.40,-0.002,63.6,113.5,92.0,21.51
2276,2017_1457_1462,2017,1457,1462,3.04,108.0,100.7,-7.3,0.029,71.2,...,4.74,6.0,99.9,111.0,11.13,0.031,67.8,115.3,100.4,14.87
2277,2017_1458_1462,2017,1458,1462,-0.70,101.1,109.6,8.4,-0.002,63.6,...,21.51,6.0,99.9,111.0,11.13,0.031,67.8,115.3,100.4,14.87


In [82]:
ss17.transpose()

,0,1,2,3,4,5,6,7,8,9,...,2268,2269,2270,2271,2272,2273,2274,2275,2276,2277
Id,2017_1112_1116,2017_1112_1124,2017_1112_1137,2017_1112_1139,2017_1112_1153,2017_1112_1166,2017_1112_1173,2017_1112_1181,2017_1112_1190,2017_1112_1195,...,2017_1452_1455,2017_1452_1457,2017_1452_1458,2017_1452_1462,2017_1455_1457,2017_1455_1458,2017_1455_1462,2017_1457_1458,2017_1457_1462,2017_1458_1462
year,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,...,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017
team1,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112,...,1452,1452,1452,1452,1455,1455,1455,1457,1457,1458
team2,1116,1124,1137,1139,1153,1166,1173,1181,1190,1195,...,1455,1457,1458,1462,1457,1458,1462,1458,1462,1462
t1_ncsos_pyth,1.88,1.88,1.88,1.88,1.88,1.88,1.88,1.88,1.88,1.88,...,-7.37,-7.37,-7.37,-7.37,NaN,NaN,NaN,3.04,3.04,-0.7
t1_sos_oppd,102.1,102.1,102.1,102.1,102.1,102.1,102.1,102.1,102.1,102.1,...,100.1,100.1,100.1,100.1,NaN,NaN,NaN,108,108,101.1
t1_sos_oppo,107.6,107.6,107.6,107.6,107.6,107.6,107.6,107.6,107.6,107.6,...,109.3,109.3,109.3,109.3,NaN,NaN,NaN,100.7,100.7,109.6
t1_sospyth,5.55,5.55,5.55,5.55,5.55,5.55,5.55,5.55,5.55,5.55,...,9.22,9.22,9.22,9.22,NaN,NaN,NaN,-7.3,-7.3,8.4
t1_luck,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,0.058,...,-0.05,-0.05,-0.05,-0.05,NaN,NaN,NaN,0.029,0.029,-0.002
t1_adjustt,65.3,65.3,65.3,65.3,65.3,65.3,65.3,65.3,65.3,65.3,...,70.8,70.8,70.8,70.8,NaN,NaN,NaN,71.2,71.2,63.6


In [80]:
test_17 = ss17[['t1_ncsos_pyth', 't1_sos_oppd', 't1_sos_oppo', 't1_sospyth', 't1_luck',
                't1_adjustt','t1_adjusto', 't1_adjustd', 't1_pyth', 't2_ncsos_pyth',
                't2_sos_oppd', 't2_sos_oppo', 't2_sospyth', 't2_luck',
                't2_adjustt','t2_adjusto', 't2_adjustd', 't2_pyth']].values

In [81]:
gb.fit(features_array17, target17)
t1_pred_prob_2017 = gb.predict_proba(test_17)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
preds17 = t1_pred_prob_2017[:,1]
ss17['pred'] = preds17

In [ ]:
ss17 = ss17[['Id', 'pred']]


In [ ]:
ss17 = ss17.sort_values('Id').reset_index(drop=True)


In [ ]:
ss17.tail()

In [ ]:
# ss17.to_csv('kp_gb_2017.csv', index=False)